In [1]:
import jax
import jax.numpy as jnp
from jax.numpy.linalg import lstsq
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch
from torch.nn.utils import parameters_to_vector, vector_to_parameters

import time
import pandas as pd
from scipy.integrate import quad
from copy import copy
import sys
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('heat_1d'), '..')))

from tedeous.data import Domain, Conditions, Equation
from tedeous.model import Model
from tedeous.callbacks import early_stopping
from tedeous.optimizers.optimizer import Optimizer
from tedeous.eval import integration

In [3]:

torch.set_default_device('cuda')
torch.set_default_dtype(torch.float64)
jax.config.update("jax_enable_x64", True)

def replace_none_with_zero(tuple_data):
    if isinstance(tuple_data, torch.Tensor):
        tuple_data[tuple_data == None] = 0
    elif tuple_data is None:
        tuple_data = torch.tensor([0.])
    elif isinstance(tuple_data, tuple):
        new_tuple = tuple(replace_none_with_zero(item) for item in tuple_data)
        return new_tuple
    return tuple_data

def gramian(net, residuals):
        # Compute the jacobian on batched data
    def jacobian():
        jac = []
        loss = residuals
        for l in loss:
            j = torch.autograd.grad(l, net.parameters(), retain_graph=True, allow_unused=True)
            j = replace_none_with_zero(j)
            j = parameters_to_vector(j).reshape(1, -1)
            jac.append(j)
        return torch.cat(jac)

    J = jacobian()
    return 1.0 / len(residuals) * J.T @ J

def grid_line_search_factory(loss, steps):

    def loss_at_step(step, model, tangent_params):
        params = parameters_to_vector(model.parameters())
        new_params = params - step*tangent_params
        vector_to_parameters(new_params, model.parameters())
        loss_val, _ = loss()
        vector_to_parameters(params, model.parameters())
        return loss_val

    def grid_line_search_update(model, tangent_params):

        losses = []
        for step in steps:
            losses.append(loss_at_step(step, model, tangent_params).reshape(1))
        losses = torch.cat(losses)
        step_size = steps[torch.argmin(losses)]

        params = parameters_to_vector(model.parameters())
        new_params = params - step_size*tangent_params
        vector_to_parameters(new_params, model.parameters())

        return step_size

    return grid_line_search_update

def exact(grid):
    return torch.exp(-torch.pi**2*grid[:,1]/4)*torch.sin(torch.pi*grid[:,0])

def heat_NGD(grid_res):
    exp_dict_list = []
    start = time.time()
    l_op = 1
    l_bound = 1
    mu = 1 / 4
    grid_steps = torch.linspace(0, 30, 31)
    steps = 0.5**grid_steps

    domain = Domain()
    domain.variable('x', [0, 1], grid_res, dtype='float64')
    domain.variable('t', [0, 1], grid_res, dtype='float64')

    boundaries = Conditions()
    x = domain.variable_dict['x']
    boundaries.dirichlet({'x': [0, 1], 't': 0}, value=torch.sin(np.pi * x))

    boundaries.dirichlet({'x': 0., 't': [0, 1]}, value=0)

    boundaries.dirichlet({'x': 1., 't': [0, 1]}, value=0)

    equation = Equation()

    heat_eq = {
        'du/dt**1':
            {
                'coeff': 1.,
                'du/dt': [1],
                'pow': 1,
                'var': 0
            },
        '-mu*d2u/dx2':
            {
                'coeff': -mu,
                'd2u/dx2': [0, 0],
                'pow': 1,
                'var': 0
            }
    }

    equation.add(heat_eq)
    
    net = torch.nn.Sequential(
        torch.nn.Linear(2, 32),
        torch.nn.Tanh(),
        torch.nn.Linear(32, 32),
        torch.nn.Tanh(),
        torch.nn.Linear(32, 1)
    )

    model = Model(net, domain, equation, boundaries)

    model.compile('autograd', lambda_operator=l_op, lambda_bound=l_bound)

    ls_update = grid_line_search_factory(model.solution_cls.evaluate, steps)

    for iteration in range(100):
        loss, _ = model.solution_cls.evaluate()
        grads = torch.autograd.grad(loss, model.net.parameters(), retain_graph=True, allow_unused=True)
        grads = replace_none_with_zero(grads)
        f_grads = parameters_to_vector(grads)

        int_res = model.solution_cls.operator._pde_compute()
        bval, true_bval, _, _ = model.solution_cls.boundary.apply_bcs()
        bound_res = bval-true_bval

        # assemble gramian
        G_int  = gramian(model.net, int_res)

        G_bdry = gramian(model.net, bound_res)
        G      = G_int + G_bdry

        # Marquardt-Levenberg
        Id = torch.eye(len(G))
        G = torch.min(torch.tensor([loss, 0.0])) * Id + G
        # compute natural gradient
        G = jnp.array(G.detach().cpu().numpy(), dtype=jnp.float64)
        f_grads =jnp.array(f_grads.detach().cpu().numpy(), dtype=jnp.float64)
        f_nat_grad = lstsq(G, f_grads)[0]
        f_nat_grad = torch.from_numpy(np.array(f_nat_grad)).to(torch.float64).to('cuda')

        # one step of NGD
        actual_step = ls_update(model.net, f_nat_grad)
        if iteration%10 == 0:
            print('iteration= ', iteration)
            print('step= ', actual_step.item())
            print('loss=' , model.solution_cls.evaluate()[0].item())
    end = time.time()
    time_NGD = end-start
    
    grid = domain.build('autograd')
    grid_test = torch.cartesian_prod(torch.linspace(0, 1, 100), torch.linspace(0, 1, 100))
    error_train = torch.sqrt(torch.mean((exact(grid).reshape(-1) - net(grid).reshape(-1)) ** 2))
    error_test = torch.sqrt(torch.mean((exact(grid_test).reshape(-1) - net(grid_test).reshape(-1)) ** 2))
    loss = model.solution_cls.evaluate()[0].detach().cpu().numpy()

    lu_f = model.solution_cls.operator.operator_compute()

    lu_f, gr = integration(lu_f, grid)

    lu_f, _ = integration(lu_f, gr)

    #########
    exp_dict_list.append({'grid_res': grid_res,
                          'error_train_NGD': error_train.item(),
                          'error_test_NGD': error_test.item(),
                          'loss_NGD': loss.item(),
                          "lu_f_NGD": lu_f.item(),
                          'time_NGD': time_NGD,
                          'type':'Heat_1d'})
    return exp_dict_list

nruns = 1
###########################
exp_dict_list = []

for grid_res in range(10, 61, 10):
    for _ in range(nruns):
        exp_dict_list.append(heat_NGD(grid_res))

exp_dict_list_flatten = [item for sublist in exp_dict_list for item in sublist]
df = pd.DataFrame(exp_dict_list_flatten)
df.to_csv('heat_1D_10_60_NGD.csv')
###########################

iteration=  0
step=  0.0009765625
loss= 0.1826040500899165
iteration=  10
step=  0.00390625
loss= 0.16827355172365294
iteration=  20
step=  0.25
loss= 1.4274715363991587e-05
iteration=  30
step=  0.125
loss= 6.5767765114063555e-12
iteration=  40
step=  0.5
loss= 6.228325084330003e-12
iteration=  50
step=  1.862645149230957e-09
loss= 6.228325067411489e-12
iteration=  60
step=  0.5
loss= 6.2283250611504075e-12
iteration=  70
step=  0.5
loss= 6.228325033082644e-12
iteration=  80
step=  0.015625
loss= 6.228325025792774e-12
iteration=  90
step=  1.0
loss= 6.228325019811698e-12
iteration=  0
step=  7.629394531249999e-06
loss= 0.1895218626340355


In [ ]:
def solver_heat(grid_res):
    exp_dict_list = []
    start = time.time()
    mu = 0.01 / np.pi

    domain = Domain()
    domain.variable('x', [-1, 1], grid_res)
    domain.variable('t', [0, 1], grid_res)

    boundaries = Conditions()
    x = domain.variable_dict['x']
    boundaries.dirichlet({'x': [-1, 1], 't': 0}, value=-torch.sin(np.pi * x))

    boundaries.dirichlet({'x': -1, 't': [0, 1]}, value=0)

    boundaries.dirichlet({'x': 1, 't': [0, 1]}, value=0)

    equation = Equation()

    burgers_eq = {
        'du/dt**1':
            {
                'coeff': 1.,
                'du/dt': [1],
                'pow': 1,
                'var': 0
            },
        '+u*du/dx':
            {
                'coeff': 1,
                'u*du/dx': [[None], [0]],
                'pow': [1, 1],
                'var': [0, 0]
            },
        '-mu*d2u/dx2':
            {
                'coeff': -mu,
                'd2u/dx2': [0, 0],
                'pow': 1,
                'var': 0
            }
    }

    equation.add(burgers_eq)

    net = torch.nn.Sequential(
        torch.nn.Linear(2, 32),
        torch.nn.Tanh(),
        torch.nn.Linear(32, 32),
        torch.nn.Tanh(),
        torch.nn.Linear(32, 1)
    )

    model = Model(net, domain, equation, boundaries)

    model.compile('autograd', lambda_operator=1/2, lambda_bound=1/2)

    cb_es = early_stopping.EarlyStopping(eps=1e-6,
                                        loss_window=100,
                                        no_improvement_patience=100,
                                        patience=2,
                                        randomize_parameter=1e-5,
                                        verbose=False)

    optim = Optimizer('Adam', {'lr': 1e-3})
    model.train(optim, 2e5, callbacks=[cb_es])
    end = time.time()

    time_adam = end - start

    grid = domain.build('autograd')

    u_exact = exact(grid).reshape(-1)

    error_adam = torch.sqrt(torch.mean((u_exact - net(grid).reshape(-1)) ** 2))

    loss_adam = model.solution_cls.evaluate()[0].detach().cpu().numpy()

    lu_f = model.solution_cls.operator.operator_compute()

    lu_f, gr = integration(lu_f, grid)

    lu_f_adam, _ = integration(lu_f, gr)

    ########

    cb_es = early_stopping.EarlyStopping(eps=1e-6,
                                        loss_window=100,
                                        no_improvement_patience=100,
                                        patience=2,
                                        randomize_parameter=1e-5,
                                        verbose=False)

    optim = Optimizer('PSO', {'pop_size': 50, #30
                                  'b': 0.4, #0.5
                                  'c2': 0.5, #0.05
                                  'c1': 0.5, 
                                  'variance': 5e-2,
                                  'lr': 1e-4})
    start = time.time()
    model.train(optim, 2e4, save_model=False, callbacks=[cb_es])
    end = time.time()
    time_pso = end - start

    u_exact = exact(grid).reshape(-1)

    error_pso = torch.sqrt(torch.mean((u_exact - net(grid).reshape(-1)) ** 2))

    loss_pso = model.solution_cls.evaluate()[0].detach().cpu().numpy()

    lu_f = model.solution_cls.operator.operator_compute()

    grid = domain.build('autograd')

    lu_f, gr = integration(lu_f, grid)

    lu_f_pso, _ = integration(lu_f, gr)

    #########

    

    exp_dict_list.append({'grid_res': grid_res,
                          'error_adam': error_adam.item(),
                          'error_PSO': error_pso.item(),
                          'loss_adam': loss_adam.item(),
                          'loss_pso': loss_pso.item(),
                          "lu_f_adam": lu_f_adam.item(),
                          "lu_f_pso": lu_f_pso.item(),
                          'time_adam': time_adam,
                          'time_pso': time_pso,
                          'type':'Burgers'})

    print('Time taken {}= {}'.format(grid_res, end - start))
    print('RMSE_adam {}= {}'.format(grid_res, error_adam))
    print('RMSE_pso {}= {}'.format(grid_res, error_pso))

    return exp_dict_list


def exact(grid):
    mu = 0.01 / np.pi

    def f(y):
        return np.exp(-np.cos(np.pi * y) / (2 * np.pi * mu))

    def integrand1(m, x, t):
        return np.sin(np.pi * (x - m)) * f(x - m) * np.exp(-m ** 2 / (4 * mu * t))

    def integrand2(m, x, t):
        return f(x - m) * np.exp(-m ** 2 / (4 * mu * t))

    def u(x, t):
        if t == 0:
            return -np.sin(np.pi * x)
        else:
            return -quad(integrand1, -np.inf, np.inf, args=(x, t))[0] / quad(integrand2, -np.inf, np.inf, args=(x, t))[
                0]

    solution = []
    for point in grid:
        solution.append(u(point[0].item(), point[1].item()))

    return torch.tensor(solution)


nruns = 5
###########################
exp_dict_list = []

for grid_res in range(50, 71, 10):
    for _ in range(nruns):
        exp_dict_list.append(solver_burgers(grid_res))

exp_dict_list_flatten = [item for sublist in exp_dict_list for item in sublist]
df = pd.DataFrame(exp_dict_list_flatten)
df.to_csv('burgers_10_100_adam_pso={}.csv')
###########################

